# Part 1 -- SQL Queries

In [1]:
import os
import sqlite3

In [2]:
con = sqlite3.connect(os.path.join('data', 'combined_data.db'))
cur = con.cursor()

In [3]:
res = cur.execute("""
    SELECT * FROM sqlite_master
""")

for c in cur.fetchall():
    print(c)

('table', 'CompanyClassification', 'CompanyClassification', 2, 'CREATE TABLE "CompanyClassification" (\n"Category" TEXT,\n  "Website" TEXT,\n  "CompanyName" TEXT,\n  "homepage_text" TEXT,\n  "h1" TEXT,\n  "h2" TEXT,\n  "h3" TEXT,\n  "nav_link_text" TEXT,\n  "meta_keywords" TEXT,\n  "meta_description" TEXT\n)')
('table', 'CompanyDataset', 'CompanyDataset', 110723, 'CREATE TABLE "CompanyDataset" (\n"Unnamed: 0" INTEGER,\n  "CompanyName" TEXT,\n  "Website" TEXT,\n  "year founded" REAL,\n  "industry" TEXT,\n  "size range" TEXT,\n  "locality" TEXT,\n  "country" TEXT,\n  "linkedin url" TEXT,\n  "current employee estimate" INTEGER,\n  "total employee estimate" INTEGER\n)')


## Query 1

In [4]:
query_1 = """
    SELECT industry, AVG(`current employee estimate`) AS average_employee_estimate 
    FROM CompanyDataset 
    WHERE `year founded` > 2002 AND `current employee estimate` > 10 
    GROUP BY industry 
    ORDER BY average_employee_estimate DESC 
    LIMIT 10;
"""

In [5]:
cur.execute(query_1)

for c in cur.fetchall():
    print(c)

('tobacco', 775.8461538461538)
('government administration', 176.92786293958522)
('supermarkets', 153.74074074074073)
('banking', 148.2915951972556)
('paper & forest products', 147.56410256410257)
('judiciary', 127.27868852459017)
('legislative office', 126.375)
('aviation & aerospace', 122.46805555555555)
('mining & metals', 120.5695652173913)
('semiconductors', 112.05504587155963)


In [6]:
cur.execute(f"""
    EXPLAIN QUERY PLAN {query_1}
""")

for c in cur.fetchall():
    print(c)

(8, 0, 0, 'SCAN CompanyDataset')
(15, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
(57, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')


### Create an index on a numeric column which will also be used in following queries

In [7]:
cur.execute("""
    CREATE INDEX employees ON CompanyDataset (`current employee estimate`);
""")

In [8]:
cur.execute(f"""
    EXPLAIN QUERY PLAN {query_1}
""")

for c in cur.fetchall():
    print(c)

(9, 0, 0, 'SEARCH CompanyDataset USING INDEX employees (current employee estimate>?)')
(16, 0, 0, 'USE TEMP B-TREE FOR GROUP BY')
(58, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')


## Query 2

In [9]:
query_2 = """
    SELECT DISTINCT d.CompanyName
    FROM CompanyDataset d, CompanyClassification c
    WHERE d.CompanyName = c.CompanyName AND 
          industry LIKE "%Technology%" AND 
          d.`current employee estimate` < 100 AND 
          TRIM(homepage_text, ' ') IS NULL
    ORDER BY d.CompanyName;
"""

In [10]:
cur.execute(query_2)

for c in cur.fetchall():
    print(c)

('anadys pharmaceuticals',)
('blubridge solutions inc',)
('boomzip cloud solutions',)
('ccw web building solutions (closed)',)
('cdn tech solutions',)
('cloud my office',)
('compudev solutions',)
('dimidium solutions inc.',)
('elogitech solutions',)
('expand learning solutions',)
('fortech software inc.',)
('global commerce & information, inc. (global ci)',)
('information services usa',)
('itrend business solutions cc',)
('kineta software',)
('kuplasolutions',)
('microtop computer co. llc',)
('midwest design computers',)
('perceptronix medical inc.',)
('risen solutions',)
('shock therapeutics biotechnologies',)
('smartphone software inc.',)
('star computers',)
('targeted sales solutions',)
('tech4sys software solutions',)
('techsys business solutions',)
('tenderfield construction software',)
('w3cloud llc',)
('wireless 1 apps inc.',)


In [11]:
cur.execute(f"""
    EXPLAIN QUERY PLAN {query_2}
""")

for c in cur.fetchall():
    print(c)

(9, 0, 0, 'SEARCH d USING INDEX employees (current employee estimate<?)')
(30, 0, 0, 'SEARCH c USING AUTOMATIC PARTIAL COVERING INDEX (CompanyName=?)')
(38, 0, 0, 'USE TEMP B-TREE FOR DISTINCT')


## Query 3

In [12]:
query_3 = """
    SELECT DISTINCT country, CompanyName, `total employee estimate`, rank_id
    FROM ( 
        SELECT country, CompanyName, `total employee estimate`, 
            ROW_NUMBER() OVER (PARTITION BY country ORDER BY `total employee estimate` DESC) AS rank_id 
        FROM CompanyDataset 
        WHERE TRIM(country, ' ') IS NOT NULL
    ) AS ranks
    WHERE rank_id <= 5
    ORDER BY country, rank_id;
"""

In [13]:
cur.execute(query_3)

for c in cur.fetchall():
    print(c)

('afghanistan', 'roshan', 986, 1)
('afghanistan', 'awcc', 823, 2)
('afghanistan', 'etisalat afghanistan', 818, 3)
('afghanistan', 'ministry of agriculture, irrigation and livestock', 508, 4)
('afghanistan', 'mtn afghanistan', 423, 5)
('albania', 'albtelecom albania', 892, 1)
('albania', 'raiffeisen bank albania', 779, 2)
('albania', 'telekom albania', 671, 3)
('albania', 'intesa sanpaolo bank albania', 447, 4)
('albania', 'national food authority', 399, 5)
('algeria', 'sonatrach', 17062, 1)
('algeria', 'algerie telecom', 3017, 2)
('algeria', 'ooredoo algérie', 2318, 3)
('algeria', 'groupe cevital', 2298, 4)
('algeria', 'naftal spa', 1879, 5)
('american samoa', 'american samoa government', 268, 1)
('american samoa', 'blue sky communications', 96, 2)
('american samoa', 'american samoa community college', 32, 3)
('american samoa', 'genexy company limited', 6, 4)
('american samoa', 'rda law firm', 3, 5)
('andorra', 'crèdit andorrà', 472, 1)
('andorra', 'banca privada andorra', 444, 2)
('an

In [14]:
cur.execute(f"""
    EXPLAIN QUERY PLAN {query_3}
""")

for c in cur.fetchall():
    print(c)

(2, 0, 0, 'CO-ROUTINE ranks')
(5, 2, 0, 'CO-ROUTINE (subquery-3)')
(8, 5, 0, 'SCAN CompanyDataset')
(21, 5, 0, 'USE TEMP B-TREE FOR ORDER BY')
(39, 2, 0, 'SCAN (subquery-3)')
(88, 0, 0, 'SCAN ranks')
(105, 0, 0, 'USE TEMP B-TREE FOR DISTINCT')
(106, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')


In [15]:
cur.execute("""
    CREATE INDEX countries ON CompanyDataset (`country`);
""")

In [16]:
cur.execute(f"""
    EXPLAIN QUERY PLAN {query_3}
""")

for c in cur.fetchall():
    print(c)

(2, 0, 0, 'CO-ROUTINE ranks')
(5, 2, 0, 'CO-ROUTINE (subquery-3)')
(9, 5, 0, 'SCAN CompanyDataset USING INDEX countries')
(30, 5, 0, 'USE TEMP B-TREE FOR RIGHT PART OF ORDER BY')
(52, 2, 0, 'SCAN (subquery-3)')
(101, 0, 0, 'SCAN ranks')
(118, 0, 0, 'USE TEMP B-TREE FOR DISTINCT')
(119, 0, 0, 'USE TEMP B-TREE FOR ORDER BY')


## Finalize

In [17]:
cur.close()
con.close()